# Load as LUME-model

Loads the base neural network model, the `sim_to_nn` transformers and the simulation variable specification from their respective files to create a [LUME-model](https://github.com/slaclab/lume-model/). The resulting instance of `TorchModel` enforces requirements on the input and output variables and can be wrapped in a `TorchModule`. The `TorchModule` can be used like a `torch.nn.Module` and is tested on a small set of simulation data.

In [1]:
import torch
import matplotlib.pyplot as plt

from lume_model.utils import variables_from_yaml
from lume_model.models import TorchModel, TorchModule

In [2]:
import os

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")  

True

In [4]:
import mlflow
import mlflow.pytorch

In [5]:
mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
mlflow.set_experiment("lcls-injector-ML")

<Experiment: artifact_location='s3://mlflow-bucket/5', creation_time=1746036982208, experiment_id='5', last_update_time=1746036982208, lifecycle_stage='active', name='lcls-injector-ML', tags={}>

In [6]:
# Ensure no active run is lingering
if mlflow.active_run():
    mlflow.end_run()

## Create model

In [8]:

with mlflow.start_run(run_name="LumeModel Run"):
    # load transformers
    input_sim_to_nn = torch.load("model/input_sim_to_nn.pt")
    output_sim_to_nn = torch.load("model/output_sim_to_nn.pt")
    # load in- and output variable specification
    input_variables, output_variables = variables_from_yaml("model/sim_variables.yml")

    # create TorchModel
    lume_model = TorchModel(
        model="model/model.pt",
        input_variables=input_variables,
        output_variables=output_variables,
        input_transformers=[input_sim_to_nn],
        output_transformers=[output_sim_to_nn],
    )
    # wrap in TorchModule
    lume_module = TorchModule(
        model=lume_model,
        input_order=lume_model.input_names,
        output_order=lume_model.output_names,
    )

    # Log basic parameters
    mlflow.log_param("model_path", "model/model.pt")
    mlflow.log_param("input_transformer", str(type(input_sim_to_nn)))
    mlflow.log_param("output_transformer", str(type(output_sim_to_nn)))
    mlflow.log_param("input_vars", input_variables)
    mlflow.log_param("output_vars", output_variables)

    # Log YAML files
    mlflow.log_artifact("model/sim_variables.yml")
    mlflow.log_artifact("model/sim_model.yml")
    mlflow.log_artifact("model/sim_module.yml")

    # load example data and calculate predictions
    inputs_small = torch.load("info/inputs_small.pt")
    outputs_small = torch.load("info/outputs_small.pt")
    with torch.no_grad():
        predictions = lume_module(inputs_small)

    # Log performance metric
    mae = torch.mean(torch.abs(predictions - outputs_small)).item()
    mlflow.log_metric("mean_absolute_error", mae)

    # Plot and save
    nrows, ncols = 3, 2
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 15))
    for i, output_name in enumerate(lume_module.output_order):
        ax_i = ax[i // ncols, i % ncols]
        if i < outputs_small.shape[1]:
            sort_idx = torch.argsort(outputs_small[:, i])
            x_axis = torch.arange(outputs_small.shape[0])
            ax_i.plot(x_axis, outputs_small[sort_idx, i], "C0x", label="outputs")
            ax_i.plot(x_axis, predictions[sort_idx, i], "C1x", label="predictions")
            ax_i.legend()
            ax_i.set_title(output_name)
    ax[-1, -1].axis('off')
    fig.tight_layout()

    plot_path = "comparison_plot_lume.png"
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)
    plt.close()


2025/04/30 14:08:29 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/04/30 14:08:29 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Loaded PyTorch model from file: model/model.pt


2025/04/30 14:08:32 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...


🏃 View run LumeModel Run at: https://ard-mlflow.slac.stanford.edu/#/experiments/5/runs/4937a5c7f598491c92f1b8bb627802d6
🧪 View experiment at: https://ard-mlflow.slac.stanford.edu/#/experiments/5


2025/04/30 14:08:32 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
